In [2]:
import sys
import os

# Add the path to `database_scripts` folder
sys.path.append(os.path.abspath("../database_scripts"))
import  db_utils as du
import create_tables as ct


conn = ct.create_or_open_database()
df_fahrrad_Zähler = du.fetch_data_df('Standorten_Zählstelle',conn)
df_mess_Fahrrad= du.fetch_data_df('Messdaten_Fahrrad',conn)
df_bezirke= du.fetch_data_df('Bezirke',conn)
df_date = du.fetch_data_df('Date_dim',conn)
df_time = du.fetch_data_df('Time_dim',conn)
df_wetter = du.fetch_data_df('Wetter',conn)

Establish SQLite connection.


In [3]:
df_fahrrad_Zähler.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Zählstelle          33 non-null     object
 1   Bezirk              33 non-null     object
 2   Beschreibung        33 non-null     object
 3   Installationsdatum  33 non-null     object
dtypes: object(4)
memory usage: 1.2+ KB


In [4]:
df_mess_Fahrrad['TimeID'] = df_mess_Fahrrad['TimeID'].astype('int32')
df_mess_Fahrrad['DateID'] = df_mess_Fahrrad['DateID'].astype('int32')
df_mess_Fahrrad['Wert'] = df_mess_Fahrrad['Wert'].astype('float32')
df_mess_Fahrrad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Zählstelle  0 non-null      object 
 1   DateID      0 non-null      int32  
 2   TimeID      0 non-null      int32  
 3   Wert        0 non-null      float32
dtypes: float32(1), int32(2), object(1)
memory usage: 132.0+ bytes


In [5]:
df_bezirke.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Bezirk       12 non-null     object 
 1   Breitengrad  12 non-null     float64
 2   Längengrad   12 non-null     float64
 3   Geometry     12 non-null     object 
dtypes: float64(2), object(2)
memory usage: 516.0+ bytes


In [ ]:
df_wetter.columns

In [6]:
df_wetter['TimeID'] = df_wetter['TimeID'].astype('int32')
df_wetter['DateID'] = df_wetter['DateID'].astype('int32')
df_wetter['temperature_2m (°C)'] = df_wetter['temperature_2m (°C)'].astype('float32')
df_wetter['relative_humidity_2m (%)'] = df_wetter['relative_humidity_2m (%)'].astype('int32')
df_wetter['rain (mm)'] = df_wetter['rain (mm)'].astype('float32')
df_wetter['snowfall (cm)'] = df_wetter['snowfall (cm)'].astype('float32')
df_wetter['cloud_cover (%)'] = df_wetter['cloud_cover (%)'].astype('float32')
df_wetter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631008 entries, 0 to 631007
Data columns (total 8 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   DateID                    631008 non-null  int32  
 1   TimeID                    631008 non-null  int32  
 2   Bezirk                    631008 non-null  object 
 3   temperature_2m (°C)       631008 non-null  float32
 4   relative_humidity_2m (%)  631008 non-null  int32  
 5   rain (mm)                 631008 non-null  float32
 6   snowfall (cm)             631008 non-null  float32
 7   cloud_cover (%)           631008 non-null  float32
dtypes: float32(4), int32(3), object(1)
memory usage: 21.7+ MB


In [9]:
df_mess_Fahrrad.isna().sum()

Zählstelle    0
DateID        0
TimeID        0
Wert          0
dtype: int64

In [7]:
#print(df_mess_Fahrrad.isna())
df_na=df_mess_Fahrrad[df_mess_Fahrrad.isna().any(axis=1)]

#df_na['Zählstelle'].unique()

In [20]:
df_na[df_na['Zählstelle']=='14-CW-JU-O']

,Zählstelle,DateID,TimeID,Wert
901296,14-CW-JU-O,1124,0,NaN
901297,14-CW-JU-O,1124,1,NaN
901298,14-CW-JU-O,1124,2,NaN
901299,14-CW-JU-O,1124,3,NaN
901300,14-CW-JU-O,1124,4,NaN
...,...,...,...,...
901747,14-CW-JU-O,1277,19,NaN
901748,14-CW-JU-O,1277,20,NaN
901749,14-CW-JU-O,1277,21,NaN
901750,14-CW-JU-O,1277,22,NaN


In [23]:
df_date[df_date['DateID']==1124]

,DateID,Date,year,month,day,day_of_the_week,is_holiday,quarter
1123,1124,28.01.2021,2021,1,28,3,0,1


In [ ]:
df_mess_Fahrrad['TimeID'] = df_mess_Fahrrad['TimeID'].astype('int32')
df_mess_Fahrrad['DateID'] = df_mess_Fahrrad['DateID'].astype('int32')
df_mess_Fahrrad['Wert'] = df_mess_Fahrrad['Wert'].astype('float32')
df_mess_Fahrrad.info()

In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_mess_Fahrrad.isna()

In [ ]:
df_merged = df_mess_Fahrrad.merge(df_fahrrad_Zähler, on='Zählstelle', how='left')

# 2. Wetterdaten hinzufügen (über DateID und optional Bezirk)
df_merged = df_merged.merge(df_wetter, on=['DateID','TimeID', 'Bezirk'], how='left')

print(df_merged.info())
print(df_merged.head())

In [ ]:

plt.figure(figsize=(10, 6))
sns.heatmap(df_merged.isna(), cbar=False, cmap='viridis')
plt.title('Heatmap der fehlenden Werte')
plt.xlabel('Spalten')
plt.ylabel('Zeilen')
plt.show()

In [ ]:
df_bezirke

In [ ]:
df_time

In [ ]:
df_wetter.isna().sum()

In [ ]:
import pandas as pd

#Merge with Wetter
# wenn mit Durschnitt on join would be TimeID auch aber hier wir können auch nach Bezirk joinen
df_merged = pd.merge(df_mess_Fahrrad, df_wetter, on=['DateID','TimeID'],how='inner') 
df_merged.head(10)

In [ ]:
df_merged = pd.merge(df_mess_Fahrrad, df_mess_Fahrrad, on="Zählstelle", how="inner")

# Merge with Date_dim on Installationsdatum and Date
df_merged = pd.merge(df_merged, df_date, on="DateID", how="inner")

# Merge with time_dim on Installationsdatum time
df_merged = pd.merge(df_merged, df_time, on='TimeID', how="inner")


# Display the merged da
df_merged.head()